In [1]:
import requests
import polars as pl

# URL API
url = 'https://api.gs.tatneft.ru/api/v2/azs/'

# Получение данных
response = requests.get(url)
data = response.json()

# Проверка статуса ответа
if data['status'] == 'success':
    # Извлечение списка заправочных станций
    stations_data = data['data']
    
    # Извлечение информации о топливе
    fuel_data = []
    for station in stations_data:
        for fuel in station['fuel']:
            fuel_data.append({
                'station_id': station['id'],  # Добавляем station_id
                'fuel_type_id': fuel['fuel_type_id'],
                'price': fuel['price'],
                'discount_price': fuel['discount_price'],
                'updated': fuel['updated'],
                'currency_code': fuel['currency_code']
            })

    # Преобразование списка топлива в DataFrame
    df_fuel = pl.DataFrame(fuel_data)

    # Преобразование списка заправочных станций в DataFrame
    df_stations = pl.DataFrame(stations_data)

    # Объединяем df_stations и df_fuel по station_id
    df_combined = df_stations.join(df_fuel, left_on='id', right_on='station_id', how='left')

    # Удаляем ненужный столбец station_id
    df_combined = df_combined

    # Вывод обновленного DataFrame
    print(df_combined)
else:
    print("Ошибка при получении данных:", data['status'])

shape: (3_873, 17)
┌──────┬───────────┬───────────┬───────────────────────┬───┬───────┬────────────────┬──────────┬─────────────────────┐
│ id   ┆ lat       ┆ lon       ┆ region                ┆ … ┆ price ┆ discount_price ┆ updated  ┆ currency_code_right │
│ ---  ┆ ---       ┆ ---       ┆ ---                   ┆   ┆ ---   ┆ ---            ┆ ---      ┆ ---                 │
│ i64  ┆ f64       ┆ f64       ┆ str                   ┆   ┆ f64   ┆ null           ┆ f64      ┆ str                 │
╞══════╪═══════════╪═══════════╪═══════════════════════╪═══╪═══════╪════════════════╪══════════╪═════════════════════╡
│ 1328 ┆ 59.554307 ┆ 30.105292 ┆ Ленинградская область ┆ … ┆ 56.99 ┆ null           ┆ 1.7444e9 ┆ rub                 │
│ 1328 ┆ 59.554307 ┆ 30.105292 ┆ Ленинградская область ┆ … ┆ 69.08 ┆ null           ┆ 1.7444e9 ┆ rub                 │
│ 1328 ┆ 59.554307 ┆ 30.105292 ┆ Ленинградская область ┆ … ┆ 87.67 ┆ null           ┆ 1.7444e9 ┆ rub                 │
│ 1328 ┆ 59.554307 ┆ 30.10529

In [2]:
data

{'status': 'success',
 'data': [{'id': 1328,
   'lat': 59.554306796985,
   'lon': 30.105292093254,
   'region': 'Ленинградская область',
   'number': 542,
   'address': 'Ленинградская область, г. Гатчина, ул. Киевская, д. 17',
   'currency_code': 'rub',
   'actualization_date': 1744375803,
   'fuel': [{'fuel_type_id': 36,
     'price': 56.99,
     'discount_price': None,
     'updated': 1744376010.35939,
     'currency_code': 'rub'},
    {'fuel_type_id': 30,
     'price': 69.08,
     'discount_price': None,
     'updated': 1744376010.35939,
     'currency_code': 'rub'},
    {'fuel_type_id': 40,
     'price': 87.67,
     'discount_price': None,
     'updated': 1744376010.35939,
     'currency_code': 'rub'},
    {'fuel_type_id': 34,
     'price': 61.69,
     'discount_price': None,
     'updated': 1744376010.35939,
     'currency_code': 'rub'}],
   'features': [2, 4, 8, 10, 11, 14, 15, 18],
   'photos': ['https://gis-media.cloud.tatneftm.ru/media/gis_photo/azs/06f81e3b-f1ab-4d89-b868-661

In [3]:
url_fuel_types = 'https://api.gs.tatneft.ru/api/v2/azs/fuel_types/'
response_fuel_types = requests.get(url_fuel_types)
data_fuel_types = response_fuel_types.json()

In [4]:
pl.DataFrame(data_fuel_types)

status,data,message
str,struct[2],null
"""success""","{[{34,""АИ-95"",""#f03c46"",false,""АИ 95"",100,100}, {30,""ДТ"",""#131d2f"",false,""ДТ"",200,200}, … {35,""AdBlue"",""#989898"",false,""AdBlue"",2000,2000}],1.7444e9}",null


In [5]:
if data_fuel_types['status'] == 'success':
    # Извлечение списка заправочных станций
    stations_fuel_data = data_fuel_types['data']
    stations_fuel_data_list = stations_fuel_data['items']   
    fuel_types_data = []

In [8]:
df_fuel_types = pl.DataFrame(stations_fuel_data['items']).rename({"id": "fuel_type_id"})

In [9]:
itog = df_combined.join(df_fuel_types, on= 'fuel_type_id', how = 'left')

In [13]:
itog.sort('id').write_excel('2.xlsx')